In [1]:
!pip install selenium

In [2]:
from selenium import webdriver

In [10]:
# web driver 객체 생성
wd = webdriver.Chrome()

In [13]:
# 최초 한번 실행
# 브라우져의 특성상.. 네트워크로 데이터를 받아서 렌더링 --> sleep을 줘서 기다려줘야한다. 대략 1~2초
wd.get("https://www.coffeebeankorea.com/store/store.asp")

In [14]:
# 자바스크립트 실행도 페이지를 로딩하는 것이므로 동일하게 sleep을 적당히 줘서 기다려야한다.
wd.execute_script("javascript:storePop2('31')")

In [17]:
from bs4 import BeautifulSoup
html = wd.page_source # 페이지 정보를 가지고 있음 html 정보
soup = BeautifulSoup(html, 'html.parser') # html 정보를 파싱

In [44]:
soup.select('#matizCoverLayer0Content > div > div > div.store_txt > h2')[0].text

'삼성봉은사거리점'

In [41]:
tr_lists = soup.select('#matizCoverLayer0Content > div > div > div.store_txt > table > tbody > tr')
tr_lists[0].td.text.strip()

'평일 07:00-22:30 | 주말/공휴일 07:00-22:00'

In [36]:
tr_lists[1].td.text().strip

'건물뒷편주차장(기계주차식)평일 최초 30분 2,000원 / 1시간 5,000원 (구매영수증 지참시 30분 2,000원 / 1시간 3,500원)1시간 초과시 10분당 1,000원 (평일,주말 동일)'

In [37]:
sub_titles = ['영업시간','주차', '주소', '전화번호', '기타정보']
content = [tr.td.text.strip() for tr in tr_lists]
dict(zip(sub_titles,content))

{'영업시간': '평일 07:00-22:30 | 주말/공휴일 07:00-22:00',
 '주차': '건물뒷편주차장(기계주차식)평일 최초 30분 2,000원 / 1시간 5,000원 (구매영수증 지참시 30분 2,000원 / 1시간 3,500원)1시간 초과시 10분당 1,000원 (평일,주말 동일)',
 '주소': '서울시 강남구 영동대로 607 1,2층',
 '전화번호': '02-3443-5618',
 '기타정보': '식약처 인증 위생등급 매우 우수매장'}

하나의 셀로 합치기

In [64]:
wd = webdriver.Chrome()

In [65]:
from bs4 import BeautifulSoup
import time # sleep 1초
store_lists = []
for i in range(500):
    try:
        wd.get("https://www.coffeebeankorea.com/store/store.asp")
        time.sleep(1)
        wd.execute_script(f"storePop2({i})")
        time.sleep(1)
        html = wd.page_source # 페이지 정보를 가지고 있음 html 정보
        soup = BeautifulSoup(html, 'html.parser') # html 정보를 파싱
        store_name = soup.select('#matizCoverLayer0Content > div > div > div.store_txt > h2')[0].text
        tr_lists = soup.select('#matizCoverLayer0Content > div > div > div.store_txt > table > tbody > tr')
        sub_titles = ['영업시간','주차', '주소', '전화번호', '기타정보']
        content = [tr.td.text.strip() for tr in tr_lists]
        store_dict = dict(zip(sub_titles,content))
        store_dict['매장명'] = store_name
        store_lists.append(store_dict)
    except:
        print(i, end = " ")

0 1 2 4 5 7 8 9 10 11 14 17 21 22 23 24 26 27 32 33 34 36 37 39 40 42 43 46 47 48 54 56 58 60 62 64 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 29

In [107]:
import pandas as pd
df = pd.DataFrame(store_lists)
df = df[ [list(df.columns)[-1]] +  list(df.columns)[:-1] ]
df

,매장명,영업시간,주차,주소,전화번호,기타정보
0,차병원점,평일 07:00~18:00 | 토요일 07:00~15:00 | 일.공휴일 휴무,병원주차장 이용가능 (병원방문시 무료),서울시 강남구 논현로 566 강남차병원1층,02-538-7615,식약처 인증 위생등급 매우 우수매장
1,강남대로점,"월-금 06:30-23:00 | 토 07:30-23:00 | 일,공휴일 08:00-...","15,000원 이상 구매시 1시간 무료 주차 가능(건물 지하 주차장 이용, 대형 차...",서울시 서초구 강남대로 369 1층,02-588-5778,NaN
2,청담에스점,월-금 07:00~22:00 | 토 07:30~22:00 | 일/공휴일 07:30~...,"건물지하 주차가능(5,000원 이상 구매시 30분 무료지급/10,000원 이상 구매...","서울시 강남구 압구정로 461 네이처포엠빌딩B108,109호",02-548-6052,NaN
3,신사점,월-목 07:00~22:00 | 금 07:00~23:00 | 토 08:00~23:0...,주차불가,서울시 강남구 도산대로 126,02-548-2741,NaN
4,역삼점,월-금 07:00~23:00 | 토 08:00~23:00 | 일 08:00~22:00,무료주차불가,"서울시 강남구 논현로 512 지상1,2층",02-569-8051,NaN
5,양재스포타임점,"월-금 07:00~22:00 | 토 7:00~ 22:00 | 일, 공휴일 08:0...",1만원이상구매시 30분무료주차가능(주차권증정),서울시 서초구 강남대로 213 24호 지하1층,02-578-6833,NaN
6,영동점,월-금 07:00~23:00 | 토 08:00~23:00 | 일/공휴일 09:00~...,주차불가,서울 서초구 반포동 736-17 P빌딩 2층,02-3443-2096,NaN
7,도곡점,"월~금 06:30~22:00 | 토,일, 공휴일 06:30~22:00","1시간30분 무료주차가능(시간초과시,10분당500원)/건물지하주차장","서울시 강남구 언주로 30길 10,112 현대비젼21 112호",02-572-2781,NaN
8,영동고앞점,월-금 07:00~22:00 | 토 07:30~22:00 | 일/공휴일 07:30~...,1대 주차 이용공간(무료),"서울시 강남구 선릉로 749 1,2층",02-544-3794,NaN
9,압구정시티점,월-목 08:00~23:00 | 금-토 08:00~23:00 | 일/공휴일 08:0...,"발렛 주차 가능 - 기본 1시간 4,000원 이후 30분당 1,000원 추가 부과","서울시 강남구 도산대로49길 13 1층 17,18호",02-543-5922,NaN


# 두번째 방법
 - webdriver를 이용하는 방법

In [164]:
# dropdown 메뉴를 선택하고 그 메뉴증에서 특정 인덱스 번호에 해당하는 메뉴를 실행
# from selenium.webdriver.support.ui import Select
# dropdown = wd.find_element(By.XPATH, '//*[@id="localTitle"]')
dropdown = wd.find_element(By.ID, 'localTitle')
dropdown.click()
dropdown = wd.find_element(By.XPATH, '//*[@id="storeLocal"]/li[1]/a') # 1 서울 ~ 11
dropdown.click()

In [144]:
shop_lists = soup.select('ul#storeListUL > li') # #은 ID
shop_lists[0].select('div.store_txt') # .은 class

[<div class="store_txt"> <p class="name"> <span>삼성봉은사거리점<strong class="distance">34m</strong></span> <a class="btn_style6" href="javascript:storePop2('31');void(0);">자세히보기</a> </p> <p class="tag"> <span class="decaf" title="디카페인">디카페인</span><span class="soy" title="두유">두유</span><span class="parking" title="주차가능">주차가능</span><span class="wifi" title="와이파이">와이파이</span><span class="smoking" title="흡연가능">흡연가능</span><span class="iqos" title="전자담배 사용가능">전자담배 사용가능</span><span class="capsule" title="캡슐판매">캡슐판매</span><span class="cake" title="케익수령">케익수령</span><span class="delivery" title="딜리버리">딜리버리</span><span class="hallcake" title="홀케익">홀케익</span> </p> <p class="address"> <span>서울시 강남구 영동대로 607 1,2층</span><!--span class="lot"><#StoreAddress2#></span--> </p> <p class="tel"><a href="tel:02-3443-5618">02-3443-5618</a></p> </div>]

In [146]:
shop = shop_lists[0].select('div.store_txt')[0]
shop

<div class="store_txt"> <p class="name"> <span>삼성봉은사거리점<strong class="distance">34m</strong></span> <a class="btn_style6" href="javascript:storePop2('31');void(0);">자세히보기</a> </p> <p class="tag"> <span class="decaf" title="디카페인">디카페인</span><span class="soy" title="두유">두유</span><span class="parking" title="주차가능">주차가능</span><span class="wifi" title="와이파이">와이파이</span><span class="smoking" title="흡연가능">흡연가능</span><span class="iqos" title="전자담배 사용가능">전자담배 사용가능</span><span class="capsule" title="캡슐판매">캡슐판매</span><span class="cake" title="케익수령">케익수령</span><span class="delivery" title="딜리버리">딜리버리</span><span class="hallcake" title="홀케익">홀케익</span> </p> <p class="address"> <span>서울시 강남구 영동대로 607 1,2층</span><!--span class="lot"><#StoreAddress2#></span--> </p> <p class="tel"><a href="tel:02-3443-5618">02-3443-5618</a></p> </div>

In [132]:
shop_name = shop.select('span')[0]
shop_name

<span>삼성봉은사거리점<strong class="distance">34m</strong></span>

In [140]:
shop_tel = shop.select('p.tel')[0].text
shop_tel

'02-3443-5618'

정리

In [182]:
# web driver 객체 생성
# 지역검색
wd = webdriver.Chrome()
wd.get('https://www.coffeebeankorea.com/store/store.asp')
html = wd.page_source  # 페이지 정보를 가지고 있음  html정보

In [186]:
# 지역검색에 대한 링크 클릭
from selenium.webdriver.common.by import By
soup = BeautifulSoup(html, 'html.parser') # html 정보를 파싱
# selenium_el = wd.find_element(By.ID,'region_srh') # 지역검색을 클릭
selenium_el = wd.find_element(By.XPATH,'//*[@id="region_srh"]')
selenium_el.click()

In [187]:
# dropdown = wd.find_element(By.ID,'localTitle')
# dropdown.click()
# time.sleep(1)    
result = []
for i in range(1,12):
    try:
        dropdown = wd.find_element(By.ID,'localTitle') # 시도를 선택해주세요 를 클릭
        dropdown.click()
        time.sleep(1)   

        dropdown = wd.find_element(By.XPATH,f'//*[@id="storeLocal"]/li[{i}]/a')  # 1 서울 ~ 11
        dropdown.click()    
        time.sleep(1)
        
        html = wd.page_source # 현재 페이지 정보를 html로 변환
        soup = BeautifulSoup(html, 'html.parser') # html 정보를 파싱
        shop_lists = soup.select('ul#storeListUL>li') # 매장정보 리스트
        for shop in shop_lists:
            result.append(
                [shop.find('span').text.strip(),
                ','.join([tag.text.strip() for tag in shop.find('p',class_='tag').select('span') ]),
                shop.find('p',class_='address').text.strip(),
                shop.find('p',class_='tel').text.strip()
                ]
            )
    except Exception as e:
        print(i)

In [188]:
import pandas as  pd
pd.DataFrame(result)

,0,1,2,3
0,브라이튼여의도점,"디카페인,두유,와이파이,케익수령","서울특별시 영등포구 국제금융로 39, 앵커원 1층",02-952-9577
1,역삼KSA한국표준협회교육센터점,"디카페인,두유,와이파이,케익수령",서울시 강남구 역삼동 738번지 1층,02-488-5912
2,삼성본관빌딩점,"디카페인,두유,와이파이,캡슐판매,케익수령",서울특별시 중구 세종대로 67 삼성 본관빌딩 1층 후문 로비 일부,02-720-5561
3,고대안암병원신관점,"디카페인,두유,와이파이,캡슐판매,케익수령",서울특별시 성북구 고려대로 73,02-544-6823
4,압구정로데오2호점,"디카페인,두유,주차가능,와이파이,캡슐판매,케익수령",서울특별시 강남구 선릉로 157길 22 2층,02-3444-9973
...,...,...,...,...
131,부산서면점,"디카페인,두유,와이파이,케익수령,홀케익",부산 부산진구 서면로 62-1 에이스메디컬타워 1F,051-802-4282
132,부산광복동점,"디카페인,두유,와이파이,흡연가능,전자담배 사용가능,케익수령,딜리버리",부산광역시 중구 창선동 2가 8-2,051-243-9105
133,제주애월DT점,"디카페인,두유,주차가능,드라이브 스루,반려동물,홀케익",제주도 제주시 애월읍 애월해안로 560,064-799-3670
134,제주아라점,"디카페인,두유,주차가능,와이파이",제주도 제주시 연북로 511,064-758-1194


공공 API 활용 자동차데이터

In [202]:
import json
import requests
import urllib.request
import xml.etree.ElementTree as ET
url = 'http://apis.data.go.kr/1613000/pubStorgeOpenApiService/getMolitStatList'
skey = 'skey'
params ={'serviceKey' : skey, 'pageNo' : '1', 'numOfRows' : '10', 'returnType' : 'xml' }
response = requests.get(url, params=params)

# XML 데이터를 파싱하여 딕셔너리로 변환
def xml_to_dict(element):
    if len(element) == 0:  # 자식 요소가 없는 경우
        return element.text
    return {child.tag: xml_to_dict(child) for child in element}

# 바이트 데이터를 문자열로 디코딩
xml_string = response.content.decode('utf-8')
# XML 파싱
root = ET.fromstring(xml_string)
xml_to_dict(root)

{'resultCode': '200',
 'resultMsg': 'OK',
 'totalCount': '690',
 'currentCount': '10',
 'pageNo': '1',
 'perPage': '10',
 'data': {'statisticsField': '국토/도시',
  'statisticsType': '승인통계',
  'statisticsName': '국토지리정보현황',
  'formName': '국가 측량 기준점 설치현황(종료)',
  'statisticsUrl': 'https://stat.molit.go.kr/portal/cate/viewChk.do?hRsId=13&hFormId=446'}}

국토 자료 다운받기

In [204]:
# 국토 자료
wd = webdriver.Chrome()  # 드라이버 로드및 브라우져 동적으로 실행
wd.get('https://stat.molit.go.kr/portal/cate/statView.do?hRsId=3&hFormId=394&hDivEng=&month_yn=')  # 크롤링할 주소를 입력
html = wd.page_source  # 페이지 정보를 가지고 있음  html정보 

In [210]:
# 지역검색에 대한 링크 클릭
from selenium.webdriver.common.by import By
soup = BeautifulSoup(html, 'html.parser') # html 정보를 파싱
wd.find_element(By.ID,'fileDownBtn').click()
wd.find_element(By.XPATH, '//*[@id="file-download-modal"]/div[2]/div[2]/ul/li[1]/div/label').click()
wd.find_element(By.XPATH,'//*[@id="file-download-modal"]/div[2]/div[3]/button').click()

API 없이 동적 크롤링으로 네이버 뉴스 가져오기
 - 네이버 뉴스 크롤링
 - https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&listType=paper&date=20240904
   1. 기사의 href 링크 제목과 링크를 크롤링
   2. 수집한 링크를 순서대로 접근해서 상세내용 크롤링
   3. 기사와 링크 내용 순으로 데이터를 결합
   4. 결합한 데이터를 DB에 적재
   5. 기사를 검색할 수 있도록 streamlit을 구성해서 검색어로 제목을 검색하고 검색리스트를 보여준 다음
   6. 사용자가 항목을 클릭하면 저장한 상세내역을 보여준다

In [229]:
import urllib.request
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import pandas as pd

In [273]:
wd = webdriver.Chrome()
wd.get(f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=032&listType=paper&date=20240904')  # 크롤링할 주소를 입력
html = wd.page_source
from selenium.webdriver.common.by import By
soup = BeautifulSoup(html, 'html.parser') # html 정보를 파싱

In [274]:
def getMainArticle(result): # result는 list
    articles = soup.select('#main_content > div.list_body.newsflash_body > ul')
    for article in articles:
        result += [(data.text,data.attrs['href']) for data in articles[0].find_all('a')[1:]]

# articles = articles[0].select('a')[1:]
# articles[0].text, articles[0].attrs['href']

In [264]:
op_news = ['032','005','020','021','081','022','023','025','028','469']
result = []
for opn in op_news:
    wd.get(f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid={opn}&listType=paper&date=20240904')  # 크롤링할 주소를 입력
    html = wd.page_source
    from selenium.webdriver.common.by import By
    soup = BeautifulSoup(html, 'html.parser') # html 정보를 파싱
    getMainArticle(result)

In [275]:
import pandas as pd
result = []
getMainArticle(result)
df1 = pd.DataFrame(result,columns = ['title','link'])
df1.head()

,title,link
0,“나홀로 응급실 전담…못 받아준 환자가 더 걱정”,https://n.news.naver.com/mnews/article/032/000...
1,인권위 ‘사정기관 인권침해’ 권리구제율 8%로 떨어졌다,https://n.news.naver.com/mnews/article/032/000...
2,이재명·한동훈 공감 뒤 ‘지구당 부활’ 급물살···찬반 분분,https://n.news.naver.com/mnews/article/032/000...
3,여야 대표 ‘공감’에…힘 받는 ‘지구당 부활’,https://n.news.naver.com/mnews/article/032/000...
4,"내달 1일 국군의날, 임시공휴일로 지정",https://n.news.naver.com/mnews/article/032/000...


In [265]:
for link in result[:5]:
    print(link[1])

https://n.news.naver.com/mnews/article/032/0003318879
https://n.news.naver.com/mnews/article/032/0003318880
https://n.news.naver.com/mnews/article/032/0003318807
https://n.news.naver.com/mnews/article/032/0003318881
https://n.news.naver.com/mnews/article/032/0003318882


In [255]:
result

[('“나홀로 응급실 전담…못 받아준 환자가 더 걱정”',
  'https://n.news.naver.com/mnews/article/032/0003318879'),
 ('인권위 ‘사정기관 인권침해’ 권리구제율 8%로 떨어졌다',
  'https://n.news.naver.com/mnews/article/032/0003318880'),
 ('이재명·한동훈 공감 뒤 ‘지구당 부활’ 급물살···찬반 분분',
  'https://n.news.naver.com/mnews/article/032/0003318807'),
 ('여야 대표 ‘공감’에…힘 받는 ‘지구당 부활’',
  'https://n.news.naver.com/mnews/article/032/0003318881'),
 ('내달 1일 국군의날, 임시공휴일로 지정',
  'https://n.news.naver.com/mnews/article/032/0003318882'),
 ('“나홀로 응급실 전담…못 받아준 환자가 더 걱정”',
  'https://n.news.naver.com/mnews/article/032/0003318879'),
 ('인권위 ‘사정기관 인권침해’ 권리구제율 8%로 떨어졌다',
  'https://n.news.naver.com/mnews/article/032/0003318880'),
 ('이재명·한동훈 공감 뒤 ‘지구당 부활’ 급물살···찬반 분분',
  'https://n.news.naver.com/mnews/article/032/0003318807'),
 ('여야 대표 ‘공감’에…힘 받는 ‘지구당 부활’',
  'https://n.news.naver.com/mnews/article/032/0003318881'),
 ('내달 1일 국군의날, 임시공휴일로 지정',
  'https://n.news.naver.com/mnews/article/032/0003318882'),
 ('“나홀로 응급실 전담…못 받아준 환자가 더 걱정”',
  'https://n.news.naver.com/mnews/a

In [267]:
url = 'https://n.news.naver.com/mnews/article/032/0003318879'
html = urllib.request.urlopen(url)
soup = BeautifulSoup(html, 'html.parser') # html 정보를 파싱
soup.select('article')[0].text

'\n전공의 이탈 6개월째이대목동병원 가보니\n\n\n\n텅 빈 구급차 전용 주차장 정부가 추석연휴를 앞두고 응급실 진료 중단을 막기 위해 군의관 등 인력을 보강하겠다고 밝힌 3일 서울 시내 한 대학병원 권역응급의료센터 앞 텅 빈 구급차 전용 주차장을 환자가 지나가고 있다. 정부는 응급실 운영에 제약이 있는 기관에 4일부터 군의관 총 15명을 파견하고, 9일부터 약 235명의 군의관과 공중보건의를 위험기관 중심으로 집중 배치할 계획이다. 조태형 기자 phototom@kyunghyang.com전공의 떠나고 남은 전문의 8명 교대로 15개 병상 지키며 버텨“전화 10번 이상 돌려야 병원행”  구급차 뺑뺑이 등 환자도 고통“1시간 동안 10군데 정도 연락을 돌려서 겨우 이 병원으로 왔어요. 의료공백 사태가 시작되고 나서는 계속 이런 식입니다.”지난 2일 오후 서울 양천구 이대목동병원 권역응급의료센터에 간신히 환자를 이송한 영등포소방서 구급대원은 “전공의 이탈 이후로는 계속 응급실 찾기가 어려워졌고, 특히 중증환자일수록 더 심한 것 같다”면서 “위급한 환자라도 응급실을 찾지 못해 결국 심정지에 처하는 경우도 생긴다”고 전했다.이대목동병원 응급실은 현재 담당 교수 한 명이 혼자 근무하고 있다. 원래 인턴 5명과 레지던트 8명 등 전공의 13명과 전문의를 포함해 총 25명의 의사가 진료를 보지만, 현재는 전문의 8명이 한 명씩 번갈아가며 진료를 보고 있다.그날 오후 처음으로 응급실 밖으로 나왔다는 남궁인 이대목동병원 응급의학과 교수는 “원래 의사가 동시에 5명 정도 있어야 하는데 지금은 저 혼자 보고 있다”며 “어쨌든 응급실 안에 들어오면 제가 책임을 지지만, 진짜 안타까운 사례는 병원이 받아주지 않아서 치료를 받지 못하는 애매한 중환자처럼 병원 밖에 더 많다고 봐야 한다”고 했다. 남궁 교수는 혼자 15개 병상을 보고 있다고 했다. 또 다른 병원 관계자 역시 “응급실 상황이 심각하다”며 “정말 교수들이 간신히 버티고 있는 것”이라고 말했다.지난 2월 정부의 의대 증원에

In [277]:
# 정규식
import re
soup = BeautifulSoup(html, 'html.parser') # html 정보를 파싱
article = soup.select('article')[0].text
re_text = re.findall(r'[가-힣0-9\s]+', article)
return ' '.join(re_text).replace('')

IndexError: list index out of range